# Tranformaciones de `output_steam_games`

## Importaciones

In [21]:
import pandas as pd
import json
import re

%load_ext autoreload
%autoreload 2
import utils

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Exploramos los datos de `output_steam_games.json`

Se cargan los datos desde el archivo json en un Dataframe y vamos familiarizandonos con la estructura de los datos.

In [6]:
# Ruta al dataset 
ruta_games = r'C:\\Users\\KAT\\Documents\\HENRY\\PI_MLops_kat\\data\\output_steam_games.json'

# Se lee de cada línea del dataset
filas_games = []
with open(ruta_games) as f:
    for line in f.readlines():
        data = json.loads(line)
        filas_games.append(data)

# Se convierte en dataframe
df_games = pd.DataFrame(filas_games)
df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


Como se puede ver existen filas llenas de nulos, las cuales procedemos a eliminarlas.

In [7]:
df_games = df_games.dropna(how='all').reset_index(drop=True)
df_games.shape

(32135, 13)

Se revisan los tipos de datos por columna y la cantidad de nulos.

In [10]:
def verificar_tipo_datos(df):
    '''
    Realiza un análisis de los tipos de datos y la presencia de valores nulos en un DataFrame.

    Esta función toma un DataFrame como entrada y devuelve un resumen que incluye información sobre
    los tipos de datos en cada columna, el porcentaje de valores no nulos y nulos, así como la
    cantidad de valores nulos por columna.
    '''

    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": [], "nulos": []}

    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].apply(type).unique())
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100-porcentaje_no_nulos, 2))
        mi_dict["nulos"].append(df[columna].isnull().sum())

    df_info = pd.DataFrame(mi_dict)
        
    return df_info

In [11]:
verificar_tipo_datos(df_games)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,publisher,"[<class 'str'>, <class 'float'>]",74.94,25.06,8052
1,genres,"[<class 'list'>, <class 'float'>]",89.78,10.22,3283
2,app_name,"[<class 'str'>, <class 'float'>]",99.99,0.01,2
3,title,"[<class 'str'>, <class 'float'>]",93.62,6.38,2050
4,url,[<class 'str'>],100.00,0.00,0
5,release_date,"[<class 'str'>, <class 'float'>]",93.57,6.43,2067
6,tags,"[<class 'list'>, <class 'float'>]",99.49,0.51,163
7,reviews_url,"[<class 'str'>, <class 'float'>]",99.99,0.01,2
8,specs,"[<class 'list'>, <class 'float'>]",97.92,2.08,670
9,price,"[<class 'float'>, <class 'str'>]",95.71,4.29,1377


Vamos revisando los datos y su estructura

In [12]:
# se observa la primera fila de 'genres'
df_games['genres'][0]

['Action', 'Casual', 'Indie', 'Simulation', 'Strategy']

In [13]:
# se observa la primera fila de 'tags'
df_games['tags'][0]

['Strategy', 'Action', 'Indie', 'Casual', 'Simulation']

In [14]:
# se observa la 100ma fila de 'specs'
df_games['specs'][100]

['Single-player', 'Multi-player']

In [15]:
# se observa el los datos unicos en la columna 'price'
df_games['price'].unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, nan, 'Free', 10.99, 1.59, 14.99, 1.99, 59.99, 8.99,
       6.99, 7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49, 15.99, 1.25,
       24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99,
       16.99, 79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99,
       149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99,
       'Free Mod', 54.99, 74.99, 'Install Theme', 0.89, 'Third-party',
       0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99,
       10.93, 1.39, 'Free HITMAN™ Holiday Pack', 36.99, 4.49, 2.0, 4.0,
       9.0, 234.99, 1.95, 1.5, 199.0, 189.0, 6.66, 27.99, 10.49, 129.99,
       179.0, 26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 199.99,
       22.99, 320.0, 38.85, 71.7, 59.95, 995.0, 27.49, 3.39, 6.0, 19.95,
       499.99, 16.06, 4.68, 131

In [16]:
# se observa el tipo de dato en 'early_access'
df_games['early_access'].unique()

array([False, True], dtype=object)

Este conjunto contiene 13 columnas y quedó con 32135 filas luego de borrar las filas de NaN, aún contienen nulos en algunas columnas.



## Transformación de 'release_date'

Es necesario extraer el año de lanzamiento del item, para ello se hace una nueva columna con el dato si existe o con un "Dato no disponible" si no esta la fecha. Luego se elimina la columa 'release_date'.

In [20]:
# Se observan las cantidades de registros por cada fecha
df_games['release_date'].value_counts()

release_date
2012-10-16    100
2017-08-31     92
2017-09-26     89
2017-06-21     82
2017-07-25     78
             ... 
1988-04-16      1
2013-08-24      1
2011-05-07      1
2010-08-21      1
2018-10-01      1
Name: count, Length: 3582, dtype: int64

In [22]:
def obtener_anio_release(fecha):
    '''
    Extrae el año de una fecha en formato 'yyyy-mm-dd' y maneja valores nulos.

    Esta función toma como entrada una fecha en formato 'yyyy-mm-dd' y devuelve el año de la fecha si
    el dato es válido. Si la fecha es nula o inconsistente, devuelve 'Dato no disponible'.

    '''
    if pd.notna(fecha):
        if re.match(r'^\d{4}-\d{2}-\d{2}$', fecha):
            return fecha.split('-')[0]
    return 'Dato no disponible'

In [23]:
# Crea columna nueva con el año
df_games['release_anio'] = df_games['release_date'].apply(obtener_anio_release)
# elimina la columna 'release_date'
df_games = df_games.drop('release_date', axis=1)
df_games.head()

,publisher,genres,app_name,title,url,tags,reviews_url,specs,price,early_access,id,developer,release_anio
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,2018
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,2017
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,2017
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,Dato no disponible


In [24]:
df_games['release_anio'].unique()

array(['2018', '2017', 'Dato no disponible', '1997', '1998', '2016',
       '2006', '2005', '2003', '2007', '2002', '2000', '1995', '1996',
       '1994', '2001', '1993', '2004', '1999', '2008', '2009', '1992',
       '1989', '2010', '2011', '2013', '2012', '2014', '1983', '1984',
       '2015', '1990', '1988', '1991', '1985', '1982', '1987', '1981',
       '1986', '2021', '2019', '1975', '1970', '1980'], dtype=object)

## Transformación de la columna 'price'

Esta columna es muy necesaria para trabajar nuestros endpoints, existen casos de promociones o que el item es gratis,por lo que se decide reemplazar esos valores a 0. Por otra parte, esta columna tiene algunos valores nulos, pero como mas adelante será necesario hacer operaciones, se adopta imputar como 0 los valores nulos.

In [25]:
def reemplaza_a_flotante(value):
    '''
    Reemplaza valores no numéricos y nulos en una columna con 0.0.

    Esta función toma un valor como entrada y trata de convertirlo a un número float.
    Si la conversión es exitosa, el valor numérico se mantiene. Si la conversión falla o
    el valor es nulo, se devuelve 0.0 en su lugar.

    '''
    if pd.isna(value):
        return 0.0
    try:
        float_value = float(value)
        return float_value
    except:
        return 0.0

In [26]:
df_games['price'] = df_games['price'].apply(reemplaza_a_flotante)
df_games['price'].dtype

dtype('float64')

## Transformación de las columnas 'publisher', 'app_name', 'title', 'developer'

En estas columnas se puede observar valores nulos, por lo que se procedera a completarlos con valores del tipo 'Dato no disponible' para que el tipo de dato sea uno solo en la columna.

In [27]:
# SColumnas a transformar
columnas_a_completar = ['publisher', 'app_name', 'title', 'developer']
# Se rellenan los nulos
df_relleno = df_games[columnas_a_completar].fillna('Sin dato disponible')
# Se borran las columnas originales y se concatenan las rellenas con todo el dataframe
df_games = pd.concat([df_games.drop(columnas_a_completar, axis=1), df_relleno], axis=1)
df_games.head()

,genres,url,tags,reviews_url,specs,price,early_access,id,release_anio,publisher,app_name,title,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",http://store.steampowered.com/app/643980/Ironb...,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,643980,2018,"Making Fun, Inc.",Ironbound,Ironbound,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",http://store.steampowered.com/app/670290/Real_...,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,670290,2017,Poolians.com,Real Pool 3D - Poolians,Real Pool 3D - Poolians,Poolians.com
3,"[Action, Adventure, Casual]",http://store.steampowered.com/app/767400/2222/,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,2017,彼岸领域,弹炸人2222,弹炸人2222,彼岸领域
4,NaN,http://store.steampowered.com/app/773570/Log_C...,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,Dato no disponible,Sin dato disponible,Log Challenge,Sin dato disponible,Sin dato disponible


## Transformación columna 'genres'

'genres' esta conformada por una lista de los distintos géneros de los videojuegos. Se necesita hacer una columna con cada género, lo que provocara la expancion de las filas.

In [28]:
df_games = df_games.explode('genres')
df_games = df_games.dropna(subset=['genres'])
df_games.head()

,genres,url,tags,reviews_url,specs,price,early_access,id,release_anio,publisher,app_name,title,developer
0,Action,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Casual,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Indie,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Simulation,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Strategy,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro


##  'tags', 'specs', 'url' y 'reviews_url'

Realizando una vista previa a los EndPoints estas columnas, no se utilizarán para las consultas de la API ni para el modelo de recomendación, por lo tanto se eliminan del dataframe.

In [29]:
df_games = df_games.drop(['tags', 'specs', 'url', 'reviews_url'], axis=1)
df_games.columns

Index(['genres', 'price', 'early_access', 'id', 'release_anio', 'publisher',
       'app_name', 'title', 'developer'],
      dtype='object')

Se verifican el tipo de dato y si quedan nulos.

In [30]:
# Se verifica como quedan los tipos de datos y si hay nulos
verificar_tipo_datos(df_games)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,genres,[<class 'str'>],100.0,0.0,0
1,price,[<class 'float'>],100.0,0.0,0
2,early_access,[<class 'bool'>],100.0,0.0,0
3,id,[<class 'str'>],100.0,0.0,0
4,release_anio,[<class 'str'>],100.0,0.0,0
5,publisher,[<class 'str'>],100.0,0.0,0
6,app_name,[<class 'str'>],100.0,0.0,0
7,title,[<class 'str'>],100.0,0.0,0
8,developer,[<class 'str'>],100.0,0.0,0


## Cargamos los dataset `output_steam_games.json`

Guardamos de manera local el dataframe transformado como `steam_games_cleaned`.

In [ ]:

df_games.to_csv('output_steam_games_cleaned.csv', index=False, encoding='utf-8')
